# Getting Started with GRASS GIS

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

This workshop is based on sections of [GIS-based Analysis of Coastal Lidar Time-Series by Hardin et al (2014)](https://link.springer.com/book/10.1007/978-1-4939-1835-5)

In this notebook we will:
* [Import Python and GRASS Python API packages](#1.-Import-Python-Packages)
* [Create a new GRASS project](#2.-Create-a-New-Project)
* [Import data](#4.-Import-Data)
* Get a quick overview of [working with GRASS Tools](#5.-GRASS-GIS-Tools) and [the Python API](#6.-GRASS-Python-API)
* [Visualize data](#7.-Data-Visualization-with-grass.jupyter)

***

## 1. Import Python Packages

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
from pathlib import Path

We are going to import the GRASS GIS Python API (`grass.script`) and the GRASS GIS Jupyter package (`grass.jupyter`), but first, we need to find the path to those packages using the `--config python_path` command. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

And now we can import the GRASS python packages!

In [ ]:
# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

***

## 2. Create a New Project

In [ ]:
!grass -e -c EPSG:3358 $HOME/csdms-grass-2024/nags_head

Starting with GRASS GIS 8.4 (now we are using 8.3+), you'll be able to do this with the `grass.script` package:

`gs.create_project("nags_head", epsg=3358, overwrite=True)`

***

## 3. Start GRASS Session

In [ ]:
gj.init("./nags_head/PERMANENT");

We've launched GRASS GIS now! We can access GRASS GIS commands using the command line interface (with the `!` line magic):

In [ ]:
!g.version

In [ ]:
!g.region -p

***

## 4. Import Data

All the data for this workshop is stored in `/data/grass-workshop`. Let's see what's there.

In [ ]:
!ls /data/grass-workshop

In [ ]:
!r.import input="/data/grass-workshop/naip_2020.tif" output="naip_2020" resolution=value resolution_value=1

In [ ]:
!g.region raster="naip_2020.1" -p

***

## 5. GRASS GIS Tools

GRASS functionality is available through tools (also called modules). There are over 500 different tools in the core distribution and over 300 addon tools or extensions that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

There is also a tool for finding other tools:

In [ ]:
!g.search.modules keyword=zonal

Here is how to get all options and flags of a GRASS tool through command line:

In [ ]:
!r.univar --help

This will open the tool's full manual page, but it will work only locally.

```
g.manual r.univar
```

For the online version of manual pages, you might visit: <https://grass.osgeo.org/grass-stable/manuals/index.html> 

***

## 6. GRASS Python API

There are two Python APIs for accessing GRASS GIS tools' functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) and [PyGRASS](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass_index.html).
PyGRASS is advantageous for more advanced workflows and low level tasks. Here, we will be using the Python Scripting Library (`import grass.script as gs`)
as it is simpler and more straightforward to use.
 

The GRASS GIS Python Scripting Library provides functions to call GRASS tools within scripts as subprocesses. The most often used functions include:

 * [run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file

Here's an example of the Python API in action:

In [ ]:
gs.run_command("g.list", type="raster")

**Try it yourself!**

_The `r.info map=NAME` command will print information about the raster NAME. Execute `r.info` in Python._

The Python API also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`
 * Run raster algebra using [mapcalc()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.mapcalc): `gs.mapcalc()`

_Try using `gs.raster_info` to print information about NAME._

Now, for a more sophisticated example. Let's import all the NAIP images using a Python for loop.

In [ ]:
files = sorted(Path('/data/grass-workshop').glob('*.tif'))

for file in files:
    name = str(file)[21:-4]
    print("importing " + name)
    gs.run_command("r.import", input=file, output=name, resolution="value", resolution_value=1)

***

## 7. Data Visualization with `grass.jupyter`

`grass.jupyter.Map()` creates and displays GRASS maps as PNG images. `gj.Map()` accepts any GRASS display module as a method by replacing the "." with "\_" in the module name. For example:

In [ ]:
example = gj.Map()
example.d_rast(map="naip_2020.1") # d.rast map=naip_2020.1
example.d_barscale(bgcolor="none") # d.barscale
example.show()

To display the image, we call the `show()` method. You can also save the image with the `save()` method.

We also might want to make this a nice square. Instead of clipping the image, we can adjust the computational region.

Here we use the `grow` parameter with a negative value to shrink the region by 100 m on each side. See the [`g.region`](https://grass.osgeo.org/grass-stable/manuals/g.region.html) manual page for more options.

In [ ]:
!g.region grow=-100 -p

Now, it's a nice rectangle!

In [ ]:
example = gj.Map(use_region=True)
example.d_rast(map="naip_2020.1") # d.rast map=naip_2020.1
example.d_barscale() # d.barscale
example.show()

Let's save the region so we can use it later.

In [ ]:
!g.region save=jockeys_ridge

**Try it yourself!**

_Modify the following code to see NAIP imagery from other years. You can also try modifying the region and see how the displayed map changes - isn't that easier than clipping?!_

In [ ]:
example = gj.Map(use_region=True)
example.d_rgb(red="naip_2020.1", green="naip_2020.2", blue="naip_2020.3")
example.d_barscale() # d.barscale
example.show()

<details>
    <summary>👉 <b>click to see an example</b></summary>
    
```python
example.d_rgb(red="naip_2014.1", green="naip_2014.2", blue="naip_2014.3")
```
</details>